In [20]:
import os
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from botocore.config import Config
import boto3
from langchain_openai import ChatOpenAI

load_dotenv()

def llm_llama():
    
    config = Config(read_timeout=120)
    AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
    AWS_ACCESS_KEY= os.getenv("AWS_ACCESS_KEY_ID")

    client = boto3.client('bedrock-runtime', region_name='us-west-2',config=config, aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY,)
 
    llm = ChatBedrock(
        client=client,
        model_id="arn:aws:bedrock:us-west-2:585008056358:inference-profile/us.meta.llama3-3-70b-instruct-v1:0",
        provider="meta",
        model_kwargs={"temperature": 0},
        region="us-west-2"
    )

    return llm
#us.meta.llama3-3-70b-instruct-v1:0
#meta.llama3-3-70b-instruct-v1:0
#mistral.mistral-large-2402-v1:0

def llm_4o():
    llm  = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))
    return llm

In [13]:
from twelvedata import TDClient
from datetime import datetime, timedelta
import pandas as pd

def get_data(interval, num_candles):
    TWELVE_DATA_API= os.getenv("TWELVE_DATA_API")
    current_time = datetime.now()
    current_time = current_time - timedelta(hours=168)
    # Format the current time to the required format
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    td = TDClient(apikey=TWELVE_DATA_API)

    ts = td.time_series(
        symbol="EUR/USD",
        interval=interval,
        outputsize=num_candles,
        timezone="America/Toronto",
        order="asc",
        end_date = formatted_time
    )

    return ts.as_pandas()


htf_df = get_data("4h", 100)
mtf_df = get_data("1h", 200)
ltf_df = get_data("15min", 100)


In [14]:
from langchain_core.prompts import ChatPromptTemplate

# HTF agent for identifying the trend bias
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
You are a specialized Forex Market Analyst focusing on higher-timeframe trends using Smart Money Concepts.
Your job is to:
1. Determine the overall trend (bullish, bearish, ranging).
2. Identify major swing points (highs/lows).
3. Summarize any significant daily order blocks or liquidity zones that are clearly visible (including swing high or low).

Format your response as:
- Overall HTF Bias: [bullish/bearish/ranging]
- Key Swing Highs and Lows: ...
- Any Notable Liquidity or Order Blocks: ...
- Brief Explanation (1–2 sentences).

Important:
- Do not invent data. If uncertain, say “Not enough info.”
- Be concise and to the point in your analysis.
"""),
        ("user", "Daily Data"),
        ("user", "{htf_df}"),
        ("user", "Please analyze this data to determine the 4h (HTF) directional bias, major swing points, and any relevant daily order blocks or liquidity pools.")
    ]
)

htf_agent_chain = prompt | llm_llama()

htf_info = htf_agent_chain.invoke({"htf_df": htf_df.to_string()})

htf_info_structured = htf_info.content

In [15]:
from langchain_core.prompts import ChatPromptTemplate

# HTF agent for identifying the trend bias
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
You are an advanced Forex Analyst specializing in Smart Money Concepts (SMC) for the 1-hour timeframe.
Your tasks:
1. Determine the current trend bias (bullish, bearish, ranging).
2. Identify 1H order blocks (bullish or bearish).
3. Mark key liquidity pools (above relative equal highs, below relative equal lows, swing high, swing low, etc.).
4. Locate fair value gaps (FVGs) if present.
5. Provide the price range of region of interest that you identify if any(might not be applicable for swing high/low). 

Format your response as:
- Current Trend Bias: [bullish/bearish/ranging]
- Identified Order Blocks: ...
- Liquidity Zones: ...
- Fair Value Gaps: ...
- Brief Explanation (1–2 sentences).

Only use the data from the “4H Data” section. If uncertain, specify “Not enough data.”
"""),
        ("user", "1H Data"),
        ("user", "{mtf_df}"),
    ]
)

mtf_agent_chain = prompt | llm_llama()

mtf_info = mtf_agent_chain.invoke({"mtf_df": mtf_df.to_string()})

mtf_info_structured = mtf_info.content

In [16]:
from langchain_core.prompts import ChatPromptTemplate

# HTF agent for identifying the trend bias
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
You are a Candlestick Pattern Identification specialist. 
Your job:
1. Read the provided chunk of 15min candle data.
2. Look specifically for patterns like Shooting Star, Pin Bars, Dojis, Engulfing Candlesor any other pattern you identify.
3. Indicate if the most recent pattern is bullish or bearish confirmation or if no pattern is found.

Output format:
 - Found Patterns: [list of patterns with candle indices/timestamps]
 - Overall Suggestion: [e.g. “Bullish Engulfing at 2025-01-10 12:00” => suggests potential buy entry]
 - Confidence or rationale

"""),
        ("user", "15min Data"),
        ("user", "{ltf_df}"),
    ]
)
ltf_agent_chain = prompt | llm_llama()

ltf_info = ltf_agent_chain.invoke({"ltf_df": ltf_df.to_string()})

ltf_info_structured = ltf_info.content


In [24]:
from pydantic import BaseModel, Field
from typing import Optional

class Schema(BaseModel):
    enter_market: bool = Field(description="This variable stores the trade decision.")
    trade_type: Optional[str] = Field(description="This variable stores the type of trade(long or short).")
    entry_price: Optional[float] = Field(description="This variable stores the entry price of the trade.")
    stop_loss: Optional[float] = Field(description="This variable stores the stop loss price of the trade.")
    take_profit: Optional[float] = Field(description="This variable stores the take profit price of the trade.")
    confident_level: Optional[int] = Field(description="This variable stores the confidence level of the trade.")
    explanation: str = Field(description="This variable stores the explanation of the trade and/or the decision.")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
You are the head trader of a Forex Hedge Fund looking for Smart Money Concept setup for EURUSD.
Your team has provided you with detailed analysis on the HTF, MTF, and LTF.
Your tasks:
- Gather and interpret the anlaysis report from the HTF, MTF, and LTF analysts.
- Understand the oerall market sentiment.
- Identify potential trade.
Here is a guide to help you to dertermine whether there is a trade opportunity:
1. Does the HTF trend align with the MTF?
2. Is the price currently sitting at a key level (e.g. Order block, fair value gap, liquidity, resistance, support)?
3. Does the LTF analysis provide a confirmation of the trade setup?
4. If stop loss is placed right under the key level, where is the next signicant key level that the price is likely to reach (take profit)?
5. What is the confidence level of this trade setup (if applicable)?
6. Provide a brief explanation of your analysis.

Please think carefully and don't forget, your objective is to make profit.
"""),
        ("user", "HTF Analysis"),
        ("user", "{htf_analysis}"),
        ("user", "MTF Analysis"),
        ("user", "{mtf_analysis}"),
        ("user", "LTF Analysis"),
        ("user", "{ltf_analysis}"),
        ("user", "Market data"),
        ("user", "{ltf_df}"),
        ("system", "If no trade opportunity is found, please avoid unncessary output.")
    ]
)

trade_agent_chain = prompt | llm_4o().with_structured_output(Schema)

trade_info = trade_agent_chain.invoke({"htf_analysis": htf_info_structured, 
                                       "mtf_analysis": mtf_info_structured, 
                                       "ltf_analysis": ltf_info_structured, 
                                       "ltf_df": ltf_df.to_string()})

trade_info_structured = trade_info
print(trade_info_structured)

enter_market=False trade_type=None entry_price=None stop_loss=None take_profit=None confident_level=None explanation='The HTF and MTF trends are bearish, but the LTF shows a potential bullish reversal pattern. This creates a conflict in the overall market sentiment. Additionally, the price is not clearly sitting at a significant key level that aligns with the bearish HTF and MTF trends. Therefore, no trade opportunity is identified at this time.'
